In [ ]:
import pandas as pd
import numpy as np
from numba import jit , cuda

import torch
from torch import nn
from sklearn.linear_model import LinearRegression

from recreate import *

def one_hot(x):
    if not isinstance(x , torch.Tensor): x = torch.Tensor(x)
    nan_index = x.isnan()
    has_nan = nan_index.any()
    x_ = torch.where(nan_index , x[~nan_index].max() + 1 , x) if has_nan else x
    dummy = torch.nn.functional.one_hot(x_.to(torch.int64)).to(torch.float)
    return dummy[...,:-1] if has_nan else dummy
gp_params = gp_parameters(True)
gp_params['device'] = torch.device("cuda")
def Ts(df):
    if isinstance(df , torch.Tensor):
        if gp_params.get('device') is not None: df = df.to(gp_params.get('device'))
        # df.share_memory_() # 执行多进程时使用：将张量移入共享内存
    return df

In [ ]:
cs_indus_code = Ts(read_gp_data('cs_indus_code',gp_params['slice_date'])[0])
size = Ts(read_gp_data('size',gp_params['slice_date'])[0])

# torch.nn.functional.one_hot(cs_indus_code.to(torch.int64) , ).to(torch.float)

In [ ]:
def _neutralize_yx(y , x_list = [] , x_group = None , no_intercept = True , index = None):
    if isinstance(x_list , torch.Tensor): x_list = [x_list]
    if len(x_list) == 0 and x_group is None: return y , None
    elif x_group is None:
        x = torch.stack(x_list,dim = -1)
    elif len(x_list) == 0:
        x = one_hot(x_group)[...,:-1]
    else:
        x = torch.stack(x_list,dim = -1)
        x = torch.cat([x,one_hot(x_group)[...,:-1]],dim=-1)
    if no_intercept: x = torch.nn.functional.pad(x , (1,0) , value = 1.)
    y = y.unsqueeze(-1)
    if index: y , x = y[index] , x[index]
    return y , x

a = _neutralize_yx(size,size,cs_indus_code)

In [ ]:
a[0].shape , a[1].shape

In [ ]:
cs_indus_code[~cs_indus_code.isnan()].max()

In [ ]:
cs_indus_code

In [ ]:
y , x_list  , x_group = size,size,cs_indus_code 
no_intercept = True 
index = None
if isinstance(x_list , torch.Tensor): x_list = [x_list]
if len(x_list) == 0 and x_group is None: 
    print(y , None) 
elif x_group is None:
    x = torch.stack(x_list,dim = -1)
elif len(x_list) == 0:
    x = one_hot(x_group)[...,:-1]
else:
    x0 = torch.stack(x_list,dim = -1)
    x1 = one_hot(x_group)[...,:-1]
    print(x0.shape , x1.shape)
    x = torch.cat([x0,x1],dim=-1)

In [ ]:
x